# Import Library

In [1]:
from tqdm import tqdm # library to show progress bar
import pickle # library to load/dump binary file

# Global Function

In [2]:
def readFile(filePath):
    f = open(filePath, "rb") #สั่งเปิดไฟล์
    file = pickle.load(f) # load ไฟล์
    f.close() #สั่งปิดไฟล์
    
    return file

In [3]:
def printPath(path):
    for i in path:
        print(f"{i['eng']:<40}{i['thai']:<40}{i['choice']:<20}")

# Import dictionary thai-english & english-thai

In [4]:
thai_dic = readFile("./dictionary/thai_test_set_dictionary.pkl") # อ่านไฟล์ thai dictionary
eng_dic = readFile("./dictionary/eng_test_set_dictionary_new.pkl") # อ่านไฟล์ english dictionary

# Import sentences thai & english

In [5]:
def readSentences(filePath, NER=False):

    file = readFile(filePath)
    
    if NER: # case ที่ มี NER tag ในข้อมูลด้วย
        sents = []
        ners = []
        for i in file:
            sent = []
            ner = []
            for j in i:
                sent.append(j[0])
                ner.append(j[1])
            sents.append(sent)
            ners.append(ner)
        return sents, ners
    else: # case ที่ ไม่มี NER tag ในข้อมูลด้วย
        return file

# Edit distance

### With NER case (case #1)

In [6]:
def editDistDP_NER(eng_sentence, thai_sentence, eng_ner, thai_ner, findPath=False): 
    
    # initial variable
    m = len(eng_sentence)
    n = len(thai_sentence)
    dp = [[0 for x in range(n + 1)] for x in range(m + 1)] # Create a table to store results of subproblems
    if findPath: # ถ้าอยากดู path
        path = [['' for x in range(n + 1)] for x in range(m + 1)]
    
    # check empty sentence
    if (not eng_sentence) or (not thai_sentence):
        return 9999
    
    # local function
    def PathFindding(m, n):
        minimumPath = []
        a, b = m, n
        while a > 0 and b > 0:
            minimumPath.insert(0, {'eng':eng_sentence[a-1], 'thai':thai_sentence[b-1], 'choice': path[a][b]})
            choice = min(dp[a][b-1],        # Insert
                        dp[a-1][b],         # Remove
                        dp[a-1][b-1])       # Replace
            if choice == dp[a][b-1]:
                b = b-1
            elif choice == dp[a-1][b]:
                a = a-1
            elif choice == dp[a-1][b-1]:
                a = a-1
                b = b-1
            else:
                a = a-1
                b = b-1 
        return minimumPath
    
    def scoring(i, j):
        # initial variable
        score = 0 # score ของความเหมือน (โดยยิ่งน้อยยิ่งเหมือน range = [0,1])
        same_meaning = 0 # ตัวแปรเก็บจำนวนความเหมือนของความหมาย
        num_thai_meaning = 0 # จำนวนความหมาย
        num_english_meaning = 0 # จำนวนความหมาย
        
        # range_of_english
#         # size = 2
#         if (i==0 or j==0):
#             return 0, False
#         elif (i==1):
#             window = range(1,4)
#         elif (i==2):
#             window = range(1,5)
#         elif (i==(m-1)):
#             window = range(m-3,m+1)
#         elif (i==(m)):
#             window = range(m-2,m+1)
#         else:
#             window = range(i-2, i+3)
        # size = 3
        if (i==0 or j==0):
            return 0, False
        elif (i==1):
            window = range(1,5)
        elif (i==2):
            window = range(1,6)
        elif (i==3):
            window = range(1,7)
        elif (i==(m-2)):
            window = range(m-5,m+1)    
        elif (i==(m-1)):
            window = range(m-4,m+1)
        elif (i==(m)):
            window = range(m-3,m+1)
        else:
            window = range(i-3, i+4)
#         # size = 4
#         if (i==0 or j==0):
#             return 0, False
#         elif (i==1):
#             window = range(1,6)
#         elif (i==2):
#             window = range(1,7)
#         elif (i==3):
#             window = range(1,8)
#         elif (i==4):
#             window = range(1,9)
#         elif (i==(m-3)):
#             window = range(m-7,m+1) 
#         elif (i==(m-2)):
#             window = range(m-6,m+1)    
#         elif (i==(m-1)):
#             window = range(m-5,m+1)
#         elif (i==(m)):
#             window = range(m-4,m+1)
#         else:
#             window = range(i-4, i+5)
        
        copy_trick = False 
        copy = False
        for eng_index in window:
            # if a word have meaning in dictionary
            try:
                if (eng_index-1 > m-1):
                    continue
                # เก็บจำนวนความหมาย
                num_english_meaning = len(eng_dic[eng_sentence[eng_index-1].lower()])
                num_thai_meaning = len(thai_dic[thai_sentence[j-1]])

                # ถ้ามีความหมายตรงกันหมดให้คะแนนเต็มไปเลย
                if thai_dic[thai_sentence[j-1]] == eng_dic[eng_sentence[eng_index-1]]:
                    score = 0
                # ถ้าไม่เหมือนหมดจะหาจำนวนความหมายที่เหมือนแล้วนำไปคิดคะแนนต่อไป
                else:
                    for thai_meaning in thai_dic[thai_sentence[j-1]]:
                        for eng_meaning in eng_dic[eng_sentence[eng_index-1].lower()]:
                            if thai_meaning == eng_meaning:
                                same_meaning += 1
                                break

                    # ช่วงคิดคะแนน
                    # ไม่มีความหมายที่ตรงกันเลย
                    if same_meaning == 0:
                        if not copy_trick:
                            score = 1
                            copy = False
                    # มีอย่างน้อย 1 ความหมายที่ตรงกัน
                    else:
                        if not copy_trick:
                            score = 0
                            copy = True
                            copy_trig = True

            # if a word doesn't have meaning in dictionary       
            # Note: ตรงนี้อาจจะต้องปรับแก้
            except:
                try:
                    if (eng_index-1 > m-1):
                        continue
                    if (eng_sentence[eng_index-1].lower()[-3:] == 'ing'):
                        word = eng_sentence[eng_index-1].lower()[:-3]
                    elif (eng_sentence[eng_index-1].lower()[-2:] == 'ed'):
                        word = eng_sentence[eng_index-1].lower()[:-2]

                    # เก็บจำนวนความหมาย
                    num_english_meaning = len(eng_dic[word])
                    num_thai_meaning = len(thai_dic[thai_sentence[j-1]])

                    # ถ้ามีความหมายตรงกันหมดให้คะแนนเต็มไปเลย
                    if thai_dic[thai_sentence[j-1]] == eng_dic[eng_sentence[eng_index-1]]:
                        score = 0
                    # ถ้าไม่เหมือนหมดจะหาจำนวนความหมายที่เหมือนแล้วนำไปคิดคะแนนต่อไป
                    else:
                        for thai_meaning in thai_dic[thai_sentence[j-1]]:
                            for eng_meaning in eng_dic[word]:
                                if thai_meaning == eng_meaning:
                                    same_meaning += 1
                                    break

                        # ช่วงคิดคะแนน
                        # ไม่มีความหมายที่ตรงกันเลย
                        if same_meaning == 0:
                            if not copy_trick:
                                score = 1
                                copy = False
                        # มีอย่างน้อย 1 ความหมายที่ตรงกัน
                        else:
                            if not copy_trick:
                                score = 0
                                copy = True
                                copy_trig = True
                except:
                    if (eng_index-1 > m-1):
                        continue
                    if (eng_ner[eng_index-1] == thai_ner[j-1]) and eng_ner[eng_index-1] != 'O' and thai_ner[j-1] != 'O':
                        if not copy_trick:
                            score = 0
                            copy = True
                            copy_trig = True
                    else:
                        if not copy_trick:
                            score = 1
                            copy = False
                        
                
                
        return score, copy # case แค่เหมือนตัวเดียวเอาเลย
                
    # Edit distance
    for i in range(m + 1):
        for j in range(n + 1):
 
            # my score
            score, copy_check = scoring(i, j)

            # If first sentence is empty, only option is to
            # insert all words of second sentence
            if i == 0:
                dp[i][j] = j   # Min. operations = j
 
            # If second sentence is empty, only option is to
            # remove all words of second sentence
            elif j == 0:
                dp[i][j] = i   # Min. operations = i
 
            # If last word have same meaning, ignore last char
            # and recur for remaining sentence
            elif copy_check:
                dp[i][j] = dp[i-1][j-1]
                if findPath:
                    path[i][j] = 'Copy'
 
            # If last word are different meaning, consider all
            # possibilities and find minimum
            else:
                choice = min(dp[i][j-1],        # Insert
                             dp[i-1][j],        # Remove
                             dp[i-1][j-1])      # Replace
                dp[i][j] = score + choice
                    
                if findPath:
                    if choice == dp[i][j-1]:
                        path[i][j] = 'Insert'
                    elif choice == dp[i-1][j]:
                        path[i][j] = 'Remove'
                    else:
                        path[i][j] = 'Replace'
                
    if findPath:
        return dp[m][n], PathFindding(m, n)
    else:
        return dp[m][n]

# Main program

In [7]:
read_ner = True

In [8]:
thai_file_path = "./data/thai_sent_NER.data"
if read_ner:
    thai_sents, thai_ners = readSentences(thai_file_path, read_ner)
else:
    thai_sents = readSentences(thai_file_path, read_ner)

In [9]:
len(thai_sents[0])

36

In [9]:
eng_file_path = "./data/eng_sent_stanfordNER.data"
if read_ner:
    eng_sents, eng_ners = readSentences(eng_file_path, read_ner)
else:
    eng_sents = readSentences(eng_file_path, read_ner)

In [10]:
num = 0
alt_num = 0
thai_sentence = thai_sents[num]
thai_ner = thai_ners[num]
eng_sentence = eng_sents[alt_num]
eng_ner = eng_ners[alt_num]

score, path =  editDistDP_NER(eng_sentence, thai_sentence, eng_ner, thai_ner, True)

score

30

In [11]:
printPath(path)

far                                     ซ้าย                                    Copy                
left                                    สุด                                     Replace             
left                                    นายติโต คาร์นาเวียน                     Copy                
indonesian national police              ผู้บัญชาการ                             Replace             
chief                                   ตำรวจแห่งชาติอินโดนีเซีย                Replace             
tito karnavian                          ตำรวจแห่งชาติอินโดนีเซีย                Copy                
from                                    จาก                                     Replace             
from                                    ซ้าย                                    Copy                
left                                    นายโรนัลด์ เดลา โรซา                    Copy                
philippine national police              ผู้บัญชาการ                             Replace    

# Evaluate

In [12]:
import random

In [13]:
# กำหนด seed เพื่อสุ่ม 1000 ประโยคภาษาอังกฤษ
N = 1000 # จำนวนประโยคภาษาไทยที่จะสุ่ม

seed = [x for x in range(N)] 

random.seed(1999)
thai_sents_sample = random.sample(range(len(thai_sents)), N)

In [14]:
eng_sents_sample = []
for i in range(N):
    random.seed(i)
    eng_index = random.sample(range(len(eng_sents)), 1000)
    eng_index.append(thai_sents_sample[i])
    eng_index = list(set(eng_index))
    if len(eng_index) > 1000:
        if eng_index[0] == thai_sents_sample[i]:
            eng_index.pop(1)
        else:
            eng_index.pop(0)
    eng_sents_sample.append(eng_index)

In [15]:
# Check
total = 0
for i, j in zip(thai_sents_sample, eng_sents_sample):
    if i in j:
        total += 1
total 

1000

In [16]:
def evaluate(case=1):
    count = {} # เอาไว้วัด acc
    length = {}
    for eng_index, thai_index in tqdm(zip(eng_sents_sample, thai_sents_sample)):
        similarity = {}
        key_min = ''
        if case == 0:
            for index in eng_index: # index = one of eng_index
                similarity[index] = editDistDP(eng_sents[index],
                                               thai_sents[thai_index],
                                              )
        elif case == 1:
            for index in eng_index: # index = one of eng_index
                similarity[index] = editDistDP_NER(eng_sents[index],
                                                   thai_sents[thai_index],
                                                   eng_ners[index],
                                                   thai_ners[thai_index]
                                                  )
        elif case == 2:
            for index in eng_index: # index = one of eng_index
                similarity[index] = editDistDP_NP( eng_sents[index],
                                                   thai_sents[thai_index],
                                                   eng_ners[index],
                                                   thai_ners[thai_index]
                                                  )

        sorted_tuples = sorted(similarity.items(), key=lambda item: item[1])
        sorted_similarity = {}
        prev = 0
        rank = 0
        for i, v in enumerate(sorted_tuples):
            if v[1] != prev:
                sorted_similarity[v[0]] = (i, v[1])
                rank = i
            else:
                sorted_similarity[v[0]] = (rank, v[1])
            prev = v[1]

        if (sorted_similarity[thai_index][0] not in length) :
            length[sorted_similarity[thai_index][0]] = [len(eng_sents[thai_index])]
        else:
            length[sorted_similarity[thai_index][0]].append(len(eng_sents[thai_index]))
        print("Index ประโยคภาษาไทย: " + str(thai_index))
        print("ประโยค: " + ''.join(thai_sents[thai_index]))

        print("Same index: " + str(thai_index) + " Value: " + str(sorted_similarity[thai_index][1]) + " Rank: " + str(sorted_similarity[thai_index][0]))
        print("Sentence: " + ' '.join(eng_sents[thai_index]))

        print("Minimum index: " + str(list(sorted_similarity)[0]) + " Value: " + str(sorted_similarity[list(sorted_similarity)[0]][1]))
        print("Sentence: " + ' '.join(eng_sents[list(sorted_similarity)[0]]))
        print("========================================")

        pos = sorted_similarity[thai_index][0]
        if pos in count:
            count[pos] += 1
        else:
            count[pos] = 1
    return count, length

In [ ]:
count, length = evaluate(1)

1it [00:01,  1.46s/it]

Index ประโยคภาษาไทย: 122618
ประโยค: จะแนะนำของนี่ค่ะ
Same index: 122618 Value: 5 Rank: 0
Sentence: i would recommend this item
Minimum index: 636936 Value: 5
Sentence: yes that should good


2it [00:06,  2.54s/it]

Index ประโยคภาษาไทย: 371094
ประโยค: คุณไม่มีทางเลือกหลังจากนั้นเว้นแต่คุณจะเป็นหนึ่งในศิลปินที่ไม่รู้จักที่ยิ่งใหญ่ที่มาพร้อมกับคุณ
Same index: 371094 Value: 20 Rank: 709
Sentence: you really have no choice after that unless it 's from one of those great unknown artist that come along and hook you
Minimum index: 552998 Value: 16
Sentence: i have no idea den he nid ask others lo lo


3it [00:08,  2.34s/it]

...

Index ประโยคภาษาไทย: 757600
ประโยค: แบบไหนที่คนสั่งกันมากสุด
Same index: 757600 Value: 8 Rank: 20
Sentence: which is the normal kind everyone get
Minimum index: 694294 Value: 7
Sentence: may i confirm your order now


286it [25:20,  4.29s/it]

Index ประโยคภาษาไทย: 108575
ประโยค: สิ่งเดียวที่ฉันจะบ่นคือแบตเตอรี่อยู่ได้ไม่นานมากและมันง่ายสำหรับพวกเขาที่จะตื่นตอนกลางคืนถ้าไม่ได้เฝ้าแต่ฉันคิดว่าพวกมันราคาถูกกว่าเปลี่ยนหลอดไฟทั้งหมด
Same index: 108575 Value: 29 Rank: 0
Sentence: my only complaint would be the battery do n't last very long and it 's easy for them to get up during the night if not watched.but i guess they are cheaper than replacing all those darn bulb
Minimum index: 108575 Value: 29
Sentence: my only complaint would be the battery do n't last very long and it 's easy for them to get up during the night if not watched.but i guess they are cheaper than replacing all those darn bulb


287it [25:24,  4.28s/it]

Index ประโยคภาษาไทย: 69425
ประโยค: แต่ไม่เลยนี่มันสายเชื่อมสำหรับกล้องยี่ห้ออื่นซึ่งผลิตออกมาต่างกับของฉัน
Same index: 69425 Value: 16 Rank: 14
Sentence: but no this is the connector for some different brand of camera that 's made differently than mine
Minimum index: 332128 Value: 15
Sentence: so do n't make the mistake i did


In [12]:
num = 743560               
alt_num = 743560                           
thai_sentence = thai_sents[num]
thai_ner = thai_ners[num]
eng_sentence = eng_sents[num]
eng_ner = eng_ners[num]

score, path =  editDistDP_NER(eng_sentence, thai_sentence, eng_ner, thai_ner, True)

score

9

In [13]:
printPath(path)

yes                                     ใช่                                     Replace             
best                                    แล้ว                                    Replace             
best                                    มี                                      Copy                
thing                                   มี                                      Copy                
about                                   ร้าน                                    Replace             
having                                  สตาร์                                   Replace             
a                                       บัค                                     Replace             
starbucks                               ใน                                      Replace             
starbucks                               ตึก                                     Copy                
in                                      ตึก                                     Copy       

In [ ]:
sorted_tuples = sorted(count.items(), key=lambda item: item[0])
sorted_count = {k: v for k, v in sorted_tuples}

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.bar(list(sorted_count.keys())[0:20], list(sorted_count.values())[0:20])
plt.xlabel('Rank')
plt.ylabel('Amount')

In [ ]:
range0 = 0
range1_4 = 0
range5_9 = 0
range10_19 = 0
for i, v in list(sorted_count.items())[0:19]:
    if i == 0:
        range0 += v
    elif i>=1 and i<=4:
        range1_4 += v
    elif i>=5 and i<=9:
        range5_9 += v
    else:
        range10_19 += v
        
print(f"Top1:     {(range0 / N)*100:.2f} %")
print(f"Top2-5:   {(range1_4 / N)*100:.2f} %")
print(f"Top5-10:  {(range5_9 / N)*100:.2f} %")
print(f"Top11-20: {(range10_19 / N)*100:.2f} %")

In [28]:
avg_lenght = {}
for i, v in length.items():
    avg_lenght[i] = sum(v) / len

0 [5, 103, 23, 25, 10, 9, 51, 4, 13, 4, 9, 10, 13, 65, 8, 28, 4, 19, 20, 7, 64, 19, 58, 19, 17, 6, 1, 35, 107, 1, 12, 41, 6, 2, 8, 13, 119, 8, 7, 8, 6, 13, 24, 38, 11, 47, 10, 4, 67, 57, 6, 13, 31, 29, 25, 17, 18, 29, 17, 18, 9, 4, 4, 14, 24, 26, 8, 6, 11, 26, 23, 26, 15, 12, 46, 9, 241, 30, 11, 9, 7, 7, 12, 78, 7, 7, 20, 14, 122, 27, 23, 18, 8, 12, 26, 36, 2, 14, 11, 25, 23, 11, 12, 8, 4, 16, 25, 18, 61, 10, 8, 16, 19, 15, 46, 12, 17, 9, 18, 14, 2, 25, 8, 19, 1, 22, 17, 13, 7, 38, 5, 19, 58, 13, 9, 25, 7, 17, 10, 12, 26, 70, 30, 7, 21, 11, 11, 7, 6, 41, 10, 16, 6, 58, 11, 3, 3, 98, 108, 10, 7, 7, 3, 18, 39, 4, 2, 5, 7, 28, 2, 13, 28, 26, 27, 3, 6, 6, 16, 15, 3, 74, 7, 36, 12, 14, 11, 3, 4, 61, 115, 77, 6, 31, 17, 12, 23, 20, 97, 7, 33, 8, 16, 8, 21, 69, 22, 12, 13, 82, 9, 26, 59, 16, 12, 87, 8, 6, 10, 10, 16, 97, 5, 3, 5, 14, 53, 20, 27, 9, 23, 20, 17, 21, 12, 15, 11, 7, 3, 10, 5, 9, 17, 108, 8, 11, 11, 16, 26, 11, 9, 10, 6, 5, 6, 17, 63, 18, 3, 26, 15, 26, 19, 4, 17, 8, 4, 12, 21, 15